# Extract Required Experience using Regex

## Load Data

In [13]:
import pandas as pd

fileName = 'Experience_Test_Results'
excel_file = '../' + fileName + '.xlsx'

df = pd.read_excel(excel_file)

### Prepare Data

In [14]:
columns = ['Title', 'OfferDescription', 'Requirements', 'Responsibilities','Descriptions']
existing_columns = [col for col in columns if col in df.columns]

requirements = df['Requirements'].tolist()
descriptions = df[existing_columns].copy().apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).tolist()

input = [
    req if isinstance(req, str) and req != '-' else desc
    for req, desc in zip(requirements, descriptions)
]

## Evaluate

In [15]:
import re

def extract_experience(text):
    # Mapping words to numbers for 0-20 and some common multiples of ten
    words_to_numbers = {
        'zero': 0, 'one': 1, 'two': 2, 'three': 3, 'four': 4, 'five': 5, 'six': 6, 'seven': 7,
        'eight': 8, 'nine': 9, 'ten': 10, 'eleven': 11, 'twelve': 12, 'thirteen': 13, 'fourteen': 14,
        'fifteen': 15, 'sixteen': 16, 'seventeen': 17, 'eighteen': 18, 'nineteen': 19, 'twenty': 20,
        'thirty': 30, 'forty': 40, 'fifty': 50, 'sixty': 60, 'seventy': 70, 'eighty': 80, 'ninety': 90
    }
    
    # Helper function to convert word to number
    def word_to_num(word):
        return words_to_numbers.get(word.lower(), None)
    
    # Define regex patterns for various ways of expressing years of experience
    patterns = [
        r'\b(\d+|' + '|'.join(words_to_numbers.keys()) + r')\s*(?:years?|year)\s+of\s+(?:relevant\s+)?(?:work\s+)?experience\b(?!\s*up\s*to\s*\d+\s*points?)',
        r'\b(\d+|' + '|'.join(words_to_numbers.keys()) + r')\s*(?:year|years)\s+experience\s+required\b(?!\s*up\s*to\s*\d+\s*points?)',
        r'\b(\d+|' + '|'.join(words_to_numbers.keys()) + r')\s*(?:year|years)\s+minimum\s+experience\b(?!\s*up\s*to\s*\d+\s*points?)',
        r'\b(?:at\s+least|minimum)\s+(\d+|' + '|'.join(words_to_numbers.keys()) + r')\s*(?:year|years)\s+of\s+(?:relevant\s+)?(?:work\s+)?experience\b(?!\s*up\s*to\s*\d+\s*points?)',
        r'\b(\d+|' + '|'.join(words_to_numbers.keys()) + r')\s*(?:year|years)\s+of\s+professional\s+experience\b(?!\s*up\s*to\s*\d+\s*points?)',
        r'\b(?:experience\s+of\s+(\d+|' + '|'.join(words_to_numbers.keys()) + r')\s*(?:year|years))\b(?!\s*up\s*to\s*\d+\s*points?)',
        r'\b(?:\d+|' + '|'.join(words_to_numbers.keys()) + r')\s*(?:year|years)\s+required\b(?!\s*up\s*to\s*\d+\s*points?)',
        r'\b(?:at\s+least|minimum)\s+(\d+|' + '|'.join(words_to_numbers.keys()) + r')\s*(?:year|years)\s+of\s+(?:relevant\s+)?experience\b(?!\s*up\s*to\s*\d+\s*points?)',
        r'\b(?:more\s+than|over)\s+(\d+|' + '|'.join(words_to_numbers.keys()) + r')\s*(?:year|years)\s+of\s+(?:relevant\s+)?(?:work\s+)?experience\b(?!\s*up\s*to\s*\d+\s*points?)',
        r'\bfrom\s+(\d+|' + '|'.join(words_to_numbers.keys()) + r')\s+to\s+(\d+|' + '|'.join(words_to_numbers.keys()) + r')\s*(?:year|years)\s+of\s+experience\b(?!\s*up\s*to\s*\d+\s*points?)',
        r'\bbetween\s+(\d+|' + '|'.join(words_to_numbers.keys()) + r')\s+and\s+(\d+|' + '|'.join(words_to_numbers.keys()) + r')\s*(?:year|years)\s+of\s+experience\b(?!\s*up\s*to\s*\d+\s*points?)',
        r'\b(\d+|' + '|'.join(words_to_numbers.keys()) + r')\s*-\s*(\d+|' + '|'.join(words_to_numbers.keys()) + r')\s*(?:year|years)\s+of\s+experience\b(?!\s*up\s*to\s*\d+\s*points?)',
        r'\b(\d+|' + '|'.join(words_to_numbers.keys()) + r')\s*plus\s*years?\s+of\s+experience\b(?!\s*up\s*to\s*\d+\s*points?)',
        r'\bup\s+to\s+(\d+|' + '|'.join(words_to_numbers.keys()) + r')\s*(?:year|years)\s+of\s+experience\b(?!\s*up\s*to\s*\d+\s*points?)',
        r'\b(\d+|' + '|'.join(words_to_numbers.keys()) + r')\s*\+\s*years?\s+of\s+experience\b(?!\s*up\s*to\s*\d+\s*points?)',
        r'\ba\s+minimum\s+of\s+(\d+|' + '|'.join(words_to_numbers.keys()) + r')\s*(?:year|years)\s+of\s+experience\b(?!\s*up\s*to\s*\d+\s*points?)',
        r'\bfrom\s+(\d+|' + '|'.join(words_to_numbers.keys()) + r')\s*(?:-\s*)?to\s+(\d+|' + '|'.join(words_to_numbers.keys()) + r')\s*(?:year|years)\s+experience\b(?!\s*up\s*to\s*\d+\s*points?)',
        r'\bfrom\s+(\d+|' + '|'.join(words_to_numbers.keys()) + r')\s+to\s+(\d+|' + '|'.join(words_to_numbers.keys()) + r')\s*(?:year|years)\s+of\s+relevant\s+experience\b(?!\s*up\s*to\s*\d+\s*points?)',
        r'\bbetween\s+(\d+|' + '|'.join(words_to_numbers.keys()) + r')\s+and\s+(\d+|' + '|'.join(words_to_numbers.keys()) + r')\s*(?:year|years)\s+of\s+relevant\s+experience\b(?!\s*up\s*to\s*\d+\s*points?)',
        r'\b(\d+|' + '|'.join(words_to_numbers.keys()) + r')\s*-\s*(\d+|' + '|'.join(words_to_numbers.keys()) + r')\s*(?:year|years)\s+of\s+relevant\s+experience\b(?!\s*up\s*to\s*\d+\s*points?)',
    ]
    
    # Combine patterns into one regex
    combined_pattern = '|'.join(f"({pattern})" for pattern in patterns)
    
    # Compile the regex
    regex = re.compile(combined_pattern, re.IGNORECASE)
    
    # Find all matches
    matches = regex.findall(text)
    
    # Extract the years from matches
    experience_years = -1
    for match in matches:
        for item in match:
            if item.isdigit() and (experience_years == -1 or int(item) < experience_years):
                experience_years = int(item)
            else:
                if item.lower() == "none": 
                    experience_years = 0
                else:
                    num = word_to_num(item)
                    if num is not None and (experience_years == -1 or num < experience_years):
                        experience_years = num
        
    return experience_years

In [21]:
requiredYears = []
classified = 0

for text in input:
    textExperience = extract_experience(text)
    if textExperience != -1: 
        classified += 1
        requiredYears.append(textExperience)
    else:
        requiredYears.append('Undefined')
        
df['RequiredExperience'] = requiredYears
df.to_excel('../' + fileName + '.xlsx', index=False, engine='openpyxl')
        
percentage = (classified / len(requiredYears)) * 100
print(f"Classified: {percentage:.2f}%")
print(f"Undefined: {(100 - percentage):.2f}%")

print("\n ----------------------------- \n")

# Calculate accuracy
true_categories = df['Experience'].tolist()
predicted_categories = df['RequiredExperience'].tolist()

# Count correct predictions
correct_predictions = sum(1 for true, pred in zip(true_categories, predicted_categories) if true == pred)

# Calculate percentage accuracy
accuracy = (correct_predictions / len(requiredYears)) * 100

# Print the result
print(f"Accuracy: {accuracy:.2f}%")

# Filter out the entries with 'Undefined' in RequiredExperience
filtered_true_categories = []
filtered_predicted_categories = []

for true, pred in zip(true_categories, predicted_categories):
    if pred != 'Undefined':  # Ignore entries with 'Undefined'
        filtered_true_categories.append(true)
        filtered_predicted_categories.append(pred)

# Count correct predictions
filtered_correct_predictions = sum(1 for true, pred in zip(filtered_true_categories, filtered_predicted_categories) if true == pred)

# Calculate percentage accuracy
filtered_accuracy = (filtered_correct_predictions / len(filtered_predicted_categories)) * 100 if filtered_predicted_categories else 0

# Print the result
print(f"Filtered Accuracy: {filtered_accuracy:.2f}%")

print("\n ----------------------------- \n")
print("Document Annotated")

Classified: 9.71%
Undefined: 90.29%

 ----------------------------- 

Accuracy: 6.86%
Filtered Accuracy: 70.59%

 ----------------------------- 

Document Annotated
